### Step 1: Upload Files and Uncompress Archives

First, let's import the necessary modules and define a function for uncompressing files.

In [ ]:
from google.colab import files
import os, tarfile, zipfile

def uncompress_file(file_name, destination):
    if file_name.endswith('.tar.gz') or file_name.endswith('.tar'):
        with tarfile.open(file_name, 'r:*') as tar:
            tar.extractall(path=destination)
    elif file_name.endswith('.zip'):
        with zipfile.ZipFile(file_name, 'r') as zip_ref:
            zip_ref.extractall(destination)

Now, upload the files and uncompress any archives.

In [ ]:
os.makedirs('/content/uncompressed_files', exist_ok=True)

uploaded = files.upload()

for filename in uploaded.keys():
    if filename.endswith(('.tar', '.tar.gz', '.zip')):
        uncompress_file(filename, '/content/uncompressed_files')
    else:
        # Move non-archive files to the uncompressed_files directory
        os.rename(filename, f'/content/uncompressed_files/{filename}')

### Step 2: Access Google Drive Directory

Mount your Google Drive to access files from it.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Specify the directory in your Google Drive that you want to access.

In [ ]:
# Replace 'your-directory-path' with your specific directory path
drive_directory = '/content/drive/My Drive/your-directory-path'

### Step 3: Iterate Through Files and Output Content of .txt Files

Define a function to iterate through files in a directory and print the content of `.txt` files.

In [ ]:
def print_txt_file_content(directory):
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('.txt'):
                file_path = os.path.join(root, file)
                with open(file_path, 'r') as file:
                    print(f"Contents of {file_path}:\n")
                    print(file.read())
                    print("\n-----\n")

In [ ]:
print_txt_file_content('/content/uncompressed_files')
print_txt_file_content(drive_directory)